## 1. BLIP으로 이미지를 설명하는 프롬프트 1차 생성

In [1]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image


def BLIP_caption(image_path):
  # 모델 로드
  processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
  model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

  # 이미지로부터 설명 생성
  inputs = processor(images=Image.open(image_path), return_tensors="pt")

  # 프롬프트 생성
  out = model.generate(**inputs)
  caption = processor.decode(out[0], skip_special_tokens=True)

  return caption

In [6]:
# 이미지 열기
image_path = "./example.png"
caption = BLIP_caption(image_path)
print(caption)

a woman taking a picture of herself


## 2. GPT를 이용하여 이미지의 프롬프트 구체화

In [14]:
import base64
import requests

OPENAI_KEY = ""

In [22]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

def get_gpt_response(image_filename, question):
    # Path to your image
    image_path = image_filename

    # Getting the base64 string
    base64_image = encode_image(image_path)

    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {OPENAI_KEY}"
    }

    payload = {
      "model": "gpt-4o",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": question
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
              }
            }
          ]
        }
      ],
      "max_tokens": 1024
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    return response.json()['choices'][0]['message']['content']

In [ ]:
prompt_background = get_gpt_response(image_path, 'Generate the detailed prompt that represents the background of given image.(Write only about the background, not the person/thing or style.)\n'+'Please refer to the following prompt.: \n'+caption)

prompt_style = get_gpt_response(image_path, 'Generate the detailed prompt that represents the picture style of given image.(Write only about the style, not the background or person/thing.)\n'+'Please refer to the following prompt.: \n'+caption)

prompt_personthing = get_gpt_response(image_path, 'Generate the detailed prompt that represents the person/thing of given image.(Write only about the person/thing, not the background or style.)\n'+'Please refer to the following prompt.: \n'+caption)

improved_prompt = f"{prompt_background}\n{prompt_style}\n{prompt_personthing}"

print(improved_prompt)

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_KEY)

response = client.images.generate(
  model="dall-e-3",
  prompt=improved_prompt,
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url

im = Image.open(requests.get(image_url, stream=True).raw)
im